# Twitter Sarcasm & Irony Detection Project
### By Asaf Levi & Haim Elbaz 
In this project, we will build several machine learning models model to detect sarcasm and irony in tweets, and compare their performance between models and preprocessing levels. 

We will be using the scikit-learn library for building our classifiers, with NLTK for natural language processing tools.


## Dataset

We will be using a labeled dataset of tweets, built as part of the research presented in the paper "Quantitative Analysis of the Differences between Sarcasm and Irony (Klinger and Litvak, 2016)".

The dataset contains data labeled as "regular", "sarcasm", "irony" and "figurative", as tagged by the tweet's writer. For our purposes, we will unite sarcasm and irony into a single category, omitting "figurative". 

## Steps Involved

1. Data Cleaning: We will clean and preprocess the tweet data by removing emojis, URLs, usernames, etc., handling necessary text transformations.

2. Feature Extraction: We will extract relevant features from the preprocessed tweets, such as bag-of-words, TF-IDF representations, and sentiment analysis.  These features will be used as input to our machine learning model.

3. Model Training: We will use scikit-learn's machine learning algorithms - Logistic Regression, Support Vector Machine Classifier and Random Forest Classifier  to train sarcasm-irony detection models on the labeled dataset.

4. Model Evaluation: We will evaluate the performance of our models with the help of a designated test dataset that will go through the same processing steps described. We will evaluate the models' performance using various metrics, such as accuracy, precision, recall, and F1 score. 

5. Prediction: Once our model is trained and evaluated, we can use it to predict sarcasm and irony in new, unseen tweets.

## Code Structure

To keep our notebook organized, we will store custom functions in a separate `helpers.py` file. This file will contain functions for data preprocessing, feature extraction, and any other utility functions we may need throughout the project. We will import these functions as needed in our notebook.

Let's get started with importing the necessary libraries and loading the dataset!

In [1]:
# Import libraries 
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix, hstack
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from helpers import preprocessing_pipeline, count_syntactic_features

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
# Load training set
raw_training_data = pd.read_csv("data/train.csv")
training_data = preprocessing_pipeline(raw_training_data)

Since our data from twitter, it would benefit to use a twitter-designated tokenizer which recognizes hashtags, usernames, urls, and emojis. After tokenizing each tweet, we use get TF-IDF for each word in the dataset. 

It is worth mentioning that we tried using simple bag of words (CountVectorizer), but the results using TF-IDF were much better. TF-IDF takes into account the importance of the word within its tweet, which seems to be quite important when it comes to tweets, in particular in large dataset, as seen in "Sentiment Analysis on COVID Tweets: An Experimental Analysis on the Impact of Count Vectorizer and TF-IDF on Sentiment Predictions using Deep Learning Models (Raza et al., 2021)"

In [3]:
tweet_tokenizer = TweetTokenizer()
def tokenize(tweet):
    return tweet_tokenizer.tokenize(tweet)
vectorizer = TfidfVectorizer(tokenizer=tokenize)
training_data_level_1 = vectorizer.fit_transform(training_data["tweets"])

In [4]:
X = training_data_level_1
y = training_data["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [8]:
models_params = [
        (
            RandomForestClassifier(random_state=42),{
                "class_weight" : [ "balanced"], # None
                "min_samples_leaf" : [i for i in range(3, 63, 10)],
                "n_estimators" : [i for i in range(5, 115, 15)],
                "n_jobs" : [7]
            }
    ),
        (
            LinearSVC(random_state=42),{
                "C" : [0.05, 0.5, 1],
                "class_weight" : ["balanced"],
            }
    ),
        (
            LogisticRegression(random_state=42),{
                "max_iter": [125, 150],
                "class_weight" : ["balanced"],
                "n_jobs" : [7],
    })
]

In [9]:
def do_gridsearch(X_train, y_train, models_params):
    results = pd.DataFrame()
    for model, param_grid in models_params:
        gs = GridSearchCV(estimator=model, 
                            error_score='raise',
                            param_grid=param_grid,
                            scoring='f1')
        gs.fit(X=X_train, y=y_train)
        results = pd.concat([results , pd.DataFrame([
    {
    'model_type' : model,
    'parameters' : params,
    'score' : score,
    }
    for params, score in zip(gs.cv_results_["params"],gs.cv_results_["mean_test_score"],
    )])])
    return results.sort_values(by='score', ascending=False)


In [10]:
tfidf_model_results = do_gridsearch(X_train, y_train, models_params)
tfidf_model_results.to_csv("tfidf_results.csv")

In [11]:
new_columns = pd.DataFrame(columns=["neg", "neu", "pos", "compound", 'Stopwords', 'Nouns', 'Verbs', 'Adverbs', 'Adjectives', 'Pronouns', "length"])
training_data = training_data.join(new_columns)

for index, row in training_data.iterrows():
    tweet = row[0]

    scores = SentimentIntensityAnalyzer().polarity_scores(tweet)

    for sentiment, score in scores.items():
        training_data.loc[index, sentiment] = score
        
    syntax_counts = count_syntactic_features(tweet)
    for syntax, count in syntax_counts.items():
        training_data.loc[index, syntax] = count
        
    training_data.loc[index, "length"] = len(tweet)

training_data = training_data.drop_duplicates()
training_data = training_data.reset_index(drop=True)
training_data.head()

,tweets,class,neg,neu,pos,compound,Stopwords,Nouns,Verbs,Adverbs,Adjectives,Pronouns,length
0,fav moment in sepp blatter vid ( 0:20 ) : `` w...,1,0.0,0.778,0.222,0.6908,10,5,1,1,2,0,116
1,just found this while walking my human ....,1,0.0,1.0,0.0,0.0,4,2,2,0,0,0,43
2,'disrespected the wife of prophet ' - pseudo l...,1,0.217,0.652,0.13,-0.296,3,6,2,0,0,0,80
3,do you know that super yeay satisfying feeling...,1,0.0,0.704,0.296,0.8126,11,3,6,1,1,0,120
4,if you 're going to call someone ignorant and ...,1,0.234,0.766,0.0,-0.6705,9,3,4,1,3,0,104


In [12]:
processed_columns = training_data.columns
processed_training_data = training_data.copy()
tf_idf_results = vectorizer.fit_transform(training_data["tweets"])
processed_training_data.drop(columns=["tweets", "class"], inplace=True)
training_data_level_2 = csr_matrix(processed_training_data.to_numpy(dtype=np.float32))

In [13]:
X = hstack([tf_idf_results, training_data_level_2])
y = training_data["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=5432)
bow_features_model_results = do_gridsearch(X_train, y_train, models_params)
bow_features_model_results.to_csv("tfidf_sentiment_syntax.csv")

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: Conv

In [14]:
bi_vectorizer = TfidfVectorizer(tokenizer=tokenize, 
                                ngram_range=(2, 2), 
                                max_features=50000)
bigram = bi_vectorizer.fit_transform(training_data["tweets"])

In [15]:
X = hstack([tf_idf_results, training_data_level_2, bigram])
y = training_data["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=99101)
bigram_model_results = do_gridsearch(X_train, y_train, models_params)

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: Conv

In [16]:
bigram_model_results.to_csv("tfidf_sentiment_syntax_bigram.csv")

## Evaluating Process Level 1

In [27]:
raw_test_data = pd.read_csv("data/test.csv")

test_data = preprocessing_pipeline(raw_test_data)
y_test = test_data["class"]
training_data = preprocessing_pipeline(raw_training_data)
whole_dataset = pd.concat([test_data, training_data])

tweet_tokenizer = TweetTokenizer()
def tokenize(tweet):
    return tweet_tokenizer.tokenize(tweet)

vectorizer = TfidfVectorizer(tokenizer=tokenize)

vectorizer.fit(whole_dataset["tweets"])
X = vectorizer.transform(training_data["tweets"])
y = training_data["class"]

X_pred = vectorizer.transform(test_data["tweets"])

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [28]:
model = RandomForestClassifier(class_weight="balanced",
                               min_samples_leaf=3,
                               n_estimators=110,
                               n_jobs=7,
                               random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.646584,0.888520,0.814486,0.767552,0.821933
recall,0.718900,0.850784,0.814486,0.784842,0.814486
f1-score,0.680827,0.869242,0.814486,0.775035,0.817385
support,1672.000000,4403.000000,0.814486,6075.000000,6075.000000


In [29]:
model = LogisticRegression(class_weight=None,
                            max_iter=150,
                            n_jobs=7,
                            random_state=99101)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,0,1,accuracy,macro avg,weighted avg
precision,0.576654,0.938330,0.827654,0.757492,0.859029
recall,0.804805,0.834071,0.827654,0.819438,0.827654
f1-score,0.671890,0.883134,0.827654,0.777512,0.836817
support,1332.000000,4743.000000,0.827654,6075.000000,6075.000000


In [30]:
model = LinearSVC(C=0.5,
                class_weight="balanced",
                random_state=99101)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.740183,0.834440,0.805597,0.787312,0.802261
recall,0.663452,0.879280,0.805597,0.771366,0.805597
f1-score,0.699720,0.856274,0.805597,0.777997,0.802826
support,2074.000000,4001.000000,0.805597,6075.000000,6075.000000


## Evaluating Process Level 2

In [31]:
new_columns = pd.DataFrame(columns=["neg", "neu", "pos", "compound", 'Stopwords', 'Nouns', 'Verbs', 'Adverbs', 'Adjectives', 'Pronouns', "length"])
training_data = training_data.join(new_columns)

for index, row in training_data.iterrows():
    tweet = row[0]

    scores = SentimentIntensityAnalyzer().polarity_scores(tweet)

    for sentiment, score in scores.items():
        training_data.loc[index, sentiment] = score
        
    syntax_counts = count_syntactic_features(tweet)
    for syntax, count in syntax_counts.items():
        training_data.loc[index, syntax] = count
        
    training_data.loc[index, "length"] = len(tweet)

training_data = training_data.drop_duplicates()
training_data = training_data.reset_index(drop=True)
processed_columns = training_data.columns
processed_training_data = training_data.copy()
tf_idf_results = vectorizer.transform(training_data["tweets"])
processed_training_data.drop(columns=["tweets", "class"], inplace=True)
training_data_level_2 = csr_matrix(processed_training_data.to_numpy(dtype=np.float32))

In [32]:
X = hstack([tf_idf_results, training_data_level_2])
y = training_data["class"]

new_columns = pd.DataFrame(columns=["neg", "neu", "pos", "compound", 'Stopwords', 'Nouns', 'Verbs', 'Adverbs', 'Adjectives', 'Pronouns', "length"])
testing_data = test_data.join(new_columns)

for index, row in testing_data.iterrows():
    tweet = row[0]

    scores = SentimentIntensityAnalyzer().polarity_scores(tweet)

    for sentiment, score in scores.items():
        testing_data.loc[index, sentiment] = score
        
    syntax_counts = count_syntactic_features(tweet)
    for syntax, count in syntax_counts.items():
        testing_data.loc[index, syntax] = count
        
    testing_data.loc[index, "length"] = len(tweet)

testing_data = testing_data.reset_index(drop=True)

processed_columns = testing_data.columns
processed_testing_data = testing_data.copy()
tf_idf_results = vectorizer.transform(testing_data["tweets"])
processed_testing_data.drop(columns=["tweets", "class"], inplace=True)

In [33]:
X_pred = hstack([X_pred, csr_matrix(processed_testing_data.to_numpy(dtype=np.float32))])

In [34]:
model = RandomForestClassifier(class_weight="balanced",
                               min_samples_leaf=3,
                               n_estimators=110,
                               n_jobs=7,
                               random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.679935,0.839896,0.790947,0.759916,0.788840
recall,0.651882,0.856141,0.790947,0.754012,0.790947
f1-score,0.665613,0.847941,0.790947,0.756777,0.789746
support,1939.000000,4136.000000,0.790947,6075.000000,6075.000000


In [35]:
model = LogisticRegression(class_weight="balanced",
                            max_iter=125,
                            n_jobs=7,
                            random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,0,1,accuracy,macro avg,weighted avg
precision,0.736417,0.745493,0.742716,0.740955,0.741845
recall,0.560606,0.865125,0.742716,0.712866,0.742716
f1-score,0.636596,0.800866,0.742716,0.718731,0.734834
support,2442.000000,3633.000000,0.742716,6075.000000,6075.000000


In [36]:
model = LinearSVC(C=0.05,
                class_weight="balanced",
                random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,0,1,accuracy,macro avg,weighted avg
precision,0.736417,0.829459,0.800988,0.782938,0.797480
recall,0.655651,0.877101,0.800988,0.766376,0.800988
f1-score,0.693691,0.852615,0.800988,0.773153,0.797992
support,2088.000000,3987.000000,0.800988,6075.000000,6075.000000


## Evaluating Process Level 3

In [37]:
bi_vectorizer = TfidfVectorizer(tokenizer=tokenize, 
                                ngram_range=(2, 2), 
                                max_features=50000)
bi_vectorizer.fit(whole_dataset["tweets"])

train_bigram = bi_vectorizer.transform(training_data["tweets"])
test_bigram = bi_vectorizer.transform(testing_data["tweets"])
X = hstack([train_bigram, X])
X_pred = hstack([test_bigram, X_pred])

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [38]:
model = LogisticRegression(class_weight="balanced",
                            max_iter=125,
                            n_jobs=7,
                            random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,0,1,accuracy,macro avg,weighted avg
precision,0.721356,0.694260,0.702551,0.707808,0.705990
recall,0.509886,0.849637,0.702551,0.679762,0.702551
f1-score,0.597460,0.764130,0.702551,0.680795,0.691975
support,2630.000000,3445.000000,0.702551,6075.000000,6075.000000


In [39]:
model = RandomForestClassifier(class_weight="balanced",
                               min_samples_leaf=3,
                               n_estimators=80,
                               n_jobs=7,
                               random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.699301,0.839658,0.796708,0.76948,0.794005
recall,0.657895,0.863625,0.796708,0.76076,0.796708
f1-score,0.677966,0.851473,0.796708,0.76472,0.795037
support,1976.000000,4099.000000,0.796708,6075.00000,6075.000000


In [40]:
model = LinearSVC(C=0.5,
                class_weight="balanced",
                random_state=42)
model.fit(X, y)
y_pred = model.predict(X_pred)
pd.DataFrame(classification_report(y_pred, y_test, output_dict=True))

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,0,1,accuracy,macro avg,weighted avg
precision,0.555675,0.942837,0.824362,0.749256,0.861644
recall,0.810832,0.827953,0.824362,0.819392,0.824362
f1-score,0.659432,0.881668,0.824362,0.770550,0.835062
support,1274.000000,4801.000000,0.824362,6075.000000,6075.000000


array([[1],
       [1],
       [1]])